In [ ]:
import requests
import json
import os
import io
from PIL import Image
from IPython.display import display

base_url = "http://localhost:8000"
endpoints_to_test = ["/detect/house", "/detect/tree", "/detect/personf", "/detect/personm"]
image_path = r"나무_7_남_00367.jpg"

analysis_results = {
    "image_size": {"width": 0, "height": 0, "area": 0},
    "house": {"size": None, "location": None, "box_area": 0},
    "windows": [], 
    "trees": [],   
    "persons": []  
}

house_counts_template = {
    "집전체": 0, "지붕": 0, "집벽": 0, "문": 0, "창문": 0, "굴뚝": 0, 
    "연기": 0, "울타리": 0, "길": 0, "연못": 0, "산": 0, "나무": 0, 
    "꽃": 0, "잔디": 0, "태양": 0
}
tree_counts_template = {
    "나무전체": 0, "기둥": 0, "수관": 0, "가지": 0, "뿌리": 0, 
    "나뭇잎": 0, "꽃": 0, "열매": 0, "그네": 0, "동물": 0, "구름": 0, 
    "달": 0, "별": 0
}
person_counts_template = {
    "사람전체": 0, "머리": 0, "얼굴": 0, "눈": 0, "코": 0, "입": 0, 
    "귀": 0, "머리카락": 0, "목": 0, "상체": 0, "팔": 0, "손": 0, 
    "다리": 0, "발": 0, "단추": 0, "주머니": 0, "신발": 0
}

all_model_results = {} 

print(f"--- 4개 모델 전체에 대해 API 서버에 요청을 보냅니다 ---")
print(f"이미지: {image_path}\n")

try:
    original_pil_image = Image.open(image_path).convert("RGB")
    img_width, img_height = original_pil_image.size
    img_area = img_width * img_height
    analysis_results["image_size"] = {"width": img_width, "height": img_height, "area": img_area}

    with open(image_path, 'rb') as f:
        image_bytes = f.read()

    for endpoint in endpoints_to_test:
        model_name = endpoint.split('/')[-1]
        print(f"--- [Testing Model: {model_name}] ---")
        
        files = {'file': (os.path.basename(image_path), image_bytes, 'image/jpeg')}
        response = requests.post(base_url + endpoint, files=files, timeout=300)

        if response.status_code == 200:
            data = response.json()
            all_model_results[model_name] = data
            print(f"✅ [Model: {model_name}] 성공! JSON 수집 완료.")
        else:
            print(f"❌ [Model: {model_name}] 오류! Status: {response.status_code}")
            all_model_results[model_name] = []
        
        print("-" * 40 + "\n")

    print("\n--- (작업 1) 크기/위치 분석을 시작합니다. ---")

    if 'house' in all_model_results:
        for det in all_model_results['house']:
            if det['name'] == '집전체':
                h_xmin, h_ymin, h_xmax, h_ymax = det['xmin'], det['ymin'], det['xmax'], det['ymax']
                h_area = (h_xmax - h_xmin) * (h_ymax - h_ymin)
                analysis_results['house']['box_area'] = h_area 
                
                ratio = h_area / img_area
                if ratio >= 2/3: analysis_results['house']['size'] = '크다'
                elif ratio <= 1/3: analysis_results['house']['size'] = '작다'
                else: analysis_results['house']['size'] = '보통'
                
                x_center_min, x_center_max = img_width * 0.25, img_width * 0.75
                y_center_min, y_center_max = img_height * 0.25, img_height * 0.75
                
                if (h_xmin >= x_center_min and h_xmax <= x_center_max and
                    h_ymin >= y_center_min and h_ymax <= y_center_max):
                    analysis_results['house']['location'] = '중앙'
                else:
                    analysis_results['house']['location'] = '비중앙'
                break 
        
        house_area = analysis_results['house']['box_area']
        if house_area > 0: 
            for det in all_model_results['house']:
                if det['name'] == '창문':
                    w_area = (det['xmax'] - det['xmin']) * (det['ymax'] - det['ymin'])
                    ratio = w_area / house_area
                    if ratio >= 2/3: size = '크다'
                    elif ratio <= 1/3: size = '작다'
                    else: size = '보통'
                    analysis_results['windows'].append({"size": size, "area": w_area})
        
    if 'tree' in all_model_results:
        for det in all_model_results['tree']:
            if det['name'] in ['나무', '나무전체']:
                t_area = (det['xmax'] - det['xmin']) * (det['ymax'] - det['ymin'])
                ratio = t_area / img_area
                if ratio <= 0.5: size = '작다'
                else: size = '보통'
                analysis_results['trees'].append({"size": size, "area": t_area})

    person_detections_for_analysis = all_model_results.get('personf', []) + all_model_results.get('personm', [])
    for det in person_detections_for_analysis:
        if det['name'] == '사람전체':
            p_height = det['ymax'] - det['ymin']
            ratio = p_height / img_height
            if ratio >= 2/3: size = '크다'
            elif ratio <= 1/3: size = '작다'
            else: size = '보통'
            analysis_results['persons'].append({"size": size, "height": p_height})
            
    print("--- (작업 1) 분석 완료 ---")

    print("\n--- (작업 2) 개수 카운트(conf >= 0.9)를 시작합니다. ---")

    house_counts = house_counts_template.copy()
    if 'house' in all_model_results:
        for det in all_model_results['house']:
            if det['confidence'] >= 0.9:
                keyword = det['name']
                if keyword in house_counts: house_counts[keyword] += 1

    tree_counts = tree_counts_template.copy()
    if 'tree' in all_model_results:
        for det in all_model_results['tree']:
            if det['confidence'] >= 0.9:
                keyword = det['name']
                if keyword in ["새", "다람쥐"]: counting_key = "동물"
                else: counting_key = keyword
                if counting_key in tree_counts: tree_counts[counting_key] += 1

    person_counts = person_counts_template.copy()
    person_detections_for_count = all_model_results.get('personf', []) + all_model_results.get('personm', [])
    for det in person_detections_for_count:
        if det['confidence'] >= 0.9:
            keyword = det['name']
            if keyword in ["운동화", "여자구두", "남자구두"]: counting_key = "신발"
            else: counting_key = keyword
            if counting_key in person_counts: person_counts[counting_key] += 1
            
    print("--- (작업 2) 카운트 완료 ---")

    print("\n\n" + "="*40)
    print("         🌟 최종 분석 결과 (크기/위치) 🌟")
    print("="*40)
    print(json.dumps(analysis_results, indent=2, ensure_ascii=False))
    
    print("\n\n" + "="*40)
    print("      🌟 최종 카운트 결과 (신뢰도 0.9 이상) 🌟")
    print("="*40)
    print("\n--- House 카운트 ---")
    print(json.dumps(house_counts, indent=2, ensure_ascii=False))
    print("\n--- Tree 카운트 ---")
    print(json.dumps(tree_counts, indent=2, ensure_ascii=False))
    print("\n--- Person 카운트 ---")
    print(json.dumps(person_counts, indent=2, ensure_ascii=False))


except FileNotFoundError:
     print(f"\n--- ❌ 파일 없음! ---")
     print(f"'{image_path}' 파일을 찾을 수 없습니다. 경로를 확인하세요.")
except requests.exceptions.ConnectionError:
    print(f"\n--- ❌ 연결 실패! ---")
    print(f"{base_url}에 연결할 수 없습니다. FastAPI 서버를 켜주세요.")
except Exception as e:
    print(f"\n--- ❌ 알 수 없는 오류! ---")
    print(f"오류: {e}")